In [ ]:
import os
import sys
import gc
from datetime import datetime
import pandas as pd
import pandas_ta as ta
from math import pi
from bokeh.plotting import figure, show
import joblib
from scipy.stats import linregress
from bokeh.io import export_png, output_notebook, export_svg
from bokeh.resources import INLINE

In [ ]:
import platform

SHOULD_GENERATE_IMAGES = platform.system() == 'Windows'
SHOULD_GENERATE_IMAGES = False
if "SHOULD_GENERATE_IMAGES" in os.environ:
    SHOULD_GENERATE_IMAGES = True

In [ ]:
TRADE_DAY_START = 900
MINIMUM_TIME = 930
MAXIMUM_TIME = 1430

MINIMUM_DATE_DATAFRAME = 20180301
MINIMUM_DATE_TRADE = 20180315

MAX_TRAIN_DATE = 20210830
MAX_TRADE_DURATION = 24

CURRENT_TARGET = 0.8
CURRENT_STOP = 0.4

DECISION_BOUNDARY = 0.4

CURRENT_EXCHANGE = 'B3'
CURRENT_ASSET = 'WDO'
CURRENT_TIMEFRAME = '5Min'

CURRENT_DAILY_FILE_CSV = 'WDO$Daily.csv'
CURRENT_5MIN_FILE_CSV = 'WDO$M5.csv'

DATA_INPUT_DIR = os.path.join('..','00.data','input')
DATA_OUTPUT_DIR = os.path.join('..','00.data','output')

In [ ]:
if "MINIMUM_TIME" in os.environ:
    MINIMUM_TIME = int(os.environ["MINIMUM_TIME"])

if "MAXIMUM_TIME" in os.environ:
    MAXIMUM_TIME = int(os.environ["MAXIMUM_TIME"])
    
if "MINIMUM_DATE_DATAFRAME" in os.environ:
    MINIMUM_DATE_DATAFRAME = int(os.environ["MINIMUM_DATE_DATAFRAME"])
    
if "MINIMUM_DATE_TRADE" in os.environ:
    MINIMUM_DATE_TRADE = int(os.environ["MINIMUM_DATE_TRADE"])    
    
if "MAX_TRAIN_DATE" in os.environ:
    MAX_TRAIN_DATE = int(os.environ["MAX_TRAIN_DATE"])        
    
if "MAX_TRADE_DURATION" in os.environ:
    MAX_TRADE_DURATION = int(os.environ["MAX_TRADE_DURATION"])

if "CURRENT_TARGET" in os.environ:
    CURRENT_TARGET = float(os.environ["CURRENT_TARGET"])

if "CURRENT_STOP" in os.environ:
    CURRENT_STOP = float(os.environ["CURRENT_STOP"])
    
if "DECISION_BOUNDARY" in os.environ:
    DECISION_BOUNDARY = float(os.environ["DECISION_BOUNDARY"])
    
if "CURRENT_ASSET" in os.environ:
    CURRENT_ASSET = os.environ["CURRENT_ASSET"]

if "CURRENT_TIMEFRAME" in os.environ:
    CURRENT_TIMEFRAME = os.environ["CURRENT_TIMEFRAME"]
    
if "CURRENT_DAILY_FILE_CSV" in os.environ:
    CURRENT_DAILY_FILE_CSV = os.environ["CURRENT_DAILY_FILE_CSV"]
    
if "CURRENT_5MIN_FILE_CSV" in os.environ:
    CURRENT_5MIN_FILE_CSV = os.environ["CURRENT_5MIN_FILE_CSV"]
    
if "DATA_INPUT_DIR" in os.environ:
    DATA_INPUT_DIR = os.environ["DATA_INPUT_DIR"]    
    
if "DATA_OUTPUT_DIR" in os.environ:
    DATA_OUTPUT_DIR = os.environ["DATA_OUTPUT_DIR"]

In [ ]:
pd.set_option('display.max_columns', None)
output_notebook(INLINE)

In [ ]:
csv_diario = pd.read_csv(os.path.join(DATA_INPUT_DIR,CURRENT_DAILY_FILE_CSV))
csv_5Min = pd.read_csv(os.path.join(DATA_INPUT_DIR,CURRENT_5MIN_FILE_CSV))

In [ ]:
csv_diario['Date'] = csv_diario.apply(lambda x: int(x['datetime'].replace('.','')), axis=1)
csv_5Min['Date'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[0].replace('.','')), axis=1)
csv_5Min['Time'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[1].replace(':','')), axis=1)
csv_5Min['DateTime'] = csv_5Min.apply(lambda x: datetime.strptime(str(x['Date']) + ' ' + str(x['Time']), '%Y%m%d %H%M'), axis=1)
csv_5Min.set_index(pd.DatetimeIndex(csv_5Min["DateTime"]), inplace=True)

In [ ]:
csv_diario = csv_diario[csv_diario['Date'] >= MINIMUM_DATE_DATAFRAME]
csv_5Min = csv_5Min[csv_5Min['Date'] >= MINIMUM_DATE_DATAFRAME]

In [ ]:
csv_5Min['ema9'] = csv_5Min.ta.ema(length=9)
csv_5Min['ema21'] = csv_5Min.ta.ema(length=21)
csv_5Min['ema55'] = csv_5Min.ta.ema(length=55)
csv_5Min['ema144'] = csv_5Min.ta.ema(length=144)
csv_5Min['ema233'] = csv_5Min.ta.ema(length=233)
csv_5Min['vwap'] = csv_5Min.ta.vwap()

In [ ]:
csv_5Min.head()

In [ ]:
data_daily = csv_diario.to_dict('records')
data_5min = csv_5Min.to_dict('records')
daily_dates = []

ohlc_daily = {}

for current_date in data_daily:
    ohlc_daily[current_date['Date']] = current_date
    daily_dates.append(current_date['Date'])

In [ ]:
def is_short(current_close, bars, target, stop):
    correct_target = current_close - target
    correct_stop = current_close + stop
    
    for x in bars:
        if x['high'] >= correct_stop:
            return 0
        if x['low'] < correct_target:
            return 1
    return 0
    
def is_long(current_close, bars, target, stop):
    correct_target = current_close + target
    correct_stop = current_close - stop
    
    for x in bars:
        if x['high'] >= correct_target:
            return 1
        if x['low'] < correct_stop:
            return 0
    return 0


In [ ]:
def get_slope(current_bar, value_to_calculate):
    return 0
    a = []
    b = []
    for current_x in range(0,11):
        a.append(current_x)
        b.append(current_bar[f'x{current_x}_{value_to_calculate}'])
    return linregress(a,b).slope

In [ ]:
generated_bars = []
generated_bars_raw = []
last_current_date = ''
current_bar_in_date = 0

for index, _ in enumerate(data_5min):
    current_date = data_5min[index]['Date']
    current_time = data_5min[index]['Time']
    
    if last_current_date != current_date:
        last_current_date = current_date
        current_bar_in_date = 0
    else:
        current_bar_in_date += 1
    
    raw_bar = {}
    for x in ['open','high','low','close','volume','Date','Time']:
        raw_bar[x] = data_5min[index][x]

    if current_date > MAX_TRAIN_DATE:
        continue

    generated_bars_raw.append(raw_bar)
    
    if current_time < MINIMUM_TIME or \
        current_time > MAXIMUM_TIME or \
        current_date < MINIMUM_DATE_TRADE:
        continue
    
    
    current_open = data_5min[index]['open']
    current_volume = data_5min[index]['volume']
    
    previous_date = [x for x in daily_dates if x < current_date]
    previous_date = previous_date[len(previous_date)-1]
    previous_date = ohlc_daily[previous_date]
    
    current_bar = {}
    
    current_bar['current_open'] = current_open
    current_bar['current_date'] = current_date
    current_bar['current_time'] = current_time
    current_bar['current_bar_in_date']= 0
    
    current_bar['previous_date'] = previous_date['Date']
    current_bar['previous_high_real'] = previous_date['high']
    current_bar['previous_close_real'] = previous_date['close']
    current_bar['previous_low_real'] = previous_date['low']
    current_bar['previous_open_real'] = previous_date['open']    
    current_bar['previous_high'] = ((previous_date['high'] / current_open) - 1) * 100
    current_bar['previous_close'] = ((previous_date['close'] / current_open) - 1) * 100
    current_bar['previous_low'] = ((previous_date['low'] / current_open) - 1) * 100
    current_bar['previous_open'] =((previous_date['open'] / current_open) - 1) * 100
    
    current_bar['x0_vwap'] = ((data_5min[index-11]['vwap'] / current_open) - 1) * 100
    current_bar['x0_ema9'] = ((data_5min[index-11]['ema9'] / current_open) - 1) * 100
    current_bar['x0_ema21'] = ((data_5min[index-11]['ema21'] / current_open) - 1) * 100
    current_bar['x0_ema55'] = ((data_5min[index-11]['ema55'] / current_open) - 1) * 100
    current_bar['x0_ema144'] = ((data_5min[index-11]['ema144'] / current_open) - 1) * 100
    current_bar['x0_ema233'] = ((data_5min[index-11]['ema233'] / current_open) - 1) * 100    
    current_bar['x0_ema9_real'] = data_5min[index-11]['ema9'] 
    current_bar['x0_ema21_real'] = data_5min[index-11]['ema21'] 
    current_bar['x0_ema55_real'] = data_5min[index-11]['ema55'] 
    current_bar['x0_ema144_real'] = data_5min[index-11]['ema144'] 
    current_bar['x0_ema233_real'] = data_5min[index-11]['ema233'] 
    current_bar['x0_open'] = ((data_5min[index-11]['open'] / current_open) - 1) * 100
    current_bar['x0_high'] = ((data_5min[index-11]['high'] / current_open) - 1) * 100
    current_bar['x0_low'] = ((data_5min[index-11]['low'] / current_open) - 1) * 100
    current_bar['x0_close'] = ((data_5min[index-11]['close'] / current_open) - 1) * 100
    current_bar['x0_volume'] = ((data_5min[index-11]['volume'] / current_volume) - 1) * 100
    current_bar['x0_open_real'] = data_5min[index-11]['open'] 
    current_bar['x0_high_real'] = data_5min[index-11]['high'] 
    current_bar['x0_low_real'] = data_5min[index-11]['low'] 
    current_bar['x0_close_real'] = data_5min[index-11]['close'] 
    current_bar['x0_height'] = current_bar['x0_high'] - current_bar['x0_low']
    current_bar['x0_body'] = ((max([current_bar['x0_open_real'],  current_bar['x0_close_real']]) / min([current_bar['x0_open_real'],  current_bar['x0_close_real']])) - 1) * 100
    current_bar['x0_roc'] = ((current_bar['x0_open_real'] / current_bar['x0_close_real']) - 1) * 100
    current_bar['x0_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema9_close'] = ((current_bar['x0_ema9_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema21_close'] = ((current_bar['x0_ema21_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema55_close'] = ((current_bar['x0_ema55_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema144_close'] = ((current_bar['x0_ema144_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema233_close'] = ((current_bar['x0_ema233_real'] / current_bar['x0_close_real']) -1) * 100
    
    
    current_bar['x1_vwap'] = ((data_5min[index-10]['vwap'] / current_open) - 1) * 100
    current_bar['x1_ema9'] = ((data_5min[index-10]['ema9'] / current_open) - 1) * 100
    current_bar['x1_ema21'] = ((data_5min[index-10]['ema21'] / current_open) - 1) * 100
    current_bar['x1_ema55'] = ((data_5min[index-10]['ema55'] / current_open) - 1) * 100
    current_bar['x1_ema144'] = ((data_5min[index-10]['ema144'] / current_open) - 1) * 100
    current_bar['x1_ema233'] = ((data_5min[index-10]['ema233'] / current_open) - 1) * 100        
    current_bar['x1_ema9_real'] = data_5min[index-10]['ema9'] 
    current_bar['x1_ema21_real'] = data_5min[index-10]['ema21']
    current_bar['x1_ema55_real'] = data_5min[index-10]['ema55'] 
    current_bar['x1_ema144_real'] = data_5min[index-10]['ema144'] 
    current_bar['x1_ema233_real'] = data_5min[index-10]['ema233'] 
    current_bar['x1_open'] = ((data_5min[index-10]['open'] / current_open) - 1) * 100
    current_bar['x1_high'] = ((data_5min[index-10]['high'] / current_open) - 1) * 100
    current_bar['x1_low'] = ((data_5min[index-10]['low'] / current_open) - 1) * 100
    current_bar['x1_close'] = ((data_5min[index-10]['close'] / current_open) - 1) * 100
    current_bar['x1_volume'] = ((data_5min[index-10]['volume'] / current_volume) - 1) * 100
    current_bar['x1_open_real'] = data_5min[index-10]['open'] 
    current_bar['x1_high_real'] = data_5min[index-10]['high'] 
    current_bar['x1_low_real'] = data_5min[index-10]['low'] 
    current_bar['x1_close_real'] = data_5min[index-10]['close'] 
    current_bar['x1_height'] = current_bar['x1_high'] - current_bar['x1_low']
    current_bar['x1_body'] = ((max([current_bar['x1_open_real'],  current_bar['x1_close_real']]) / min([current_bar['x1_open_real'],  current_bar['x1_close_real']])) - 1) * 100
    current_bar['x1_roc'] = ((current_bar['x1_open_real'] / current_bar['x1_close_real']) - 1) * 100
    current_bar['x1_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema9_close'] = ((current_bar['x1_ema9_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema21_close'] = ((current_bar['x1_ema21_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema55_close'] = ((current_bar['x1_ema55_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema144_close'] = ((current_bar['x1_ema144_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema233_close'] = ((current_bar['x1_ema233_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_x0_close'] = ((current_bar['x1_close_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x1_x0_high'] = ((current_bar['x1_high_real'] / current_bar['x0_high_real']) -1) * 100
    current_bar['x1_x0_low'] = ((current_bar['x1_low_real'] / current_bar['x0_low_real']) -1) * 100
    
    
    current_bar['x2_vwap'] = ((data_5min[index-9]['vwap'] / current_open) - 1) * 100
    current_bar['x2_ema9'] = ((data_5min[index-9]['ema9'] / current_open) - 1) * 100
    current_bar['x2_ema21'] = ((data_5min[index-9]['ema21'] / current_open) - 1) * 100
    current_bar['x2_ema55'] = ((data_5min[index-9]['ema55'] / current_open) - 1) * 100
    current_bar['x2_ema144'] = ((data_5min[index-9]['ema144'] / current_open) - 1) * 100
    current_bar['x2_ema233'] = ((data_5min[index-9]['ema233'] / current_open) - 1) * 100        
    current_bar['x2_ema9_real'] = data_5min[index-9]['ema9'] 
    current_bar['x2_ema21_real'] = data_5min[index-9]['ema21'] 
    current_bar['x2_ema55_real'] = data_5min[index-9]['ema55'] 
    current_bar['x2_ema144_real'] = data_5min[index-9]['ema144']
    current_bar['x2_ema233_real'] = data_5min[index-9]['ema233']
    current_bar['x2_open'] = ((data_5min[index-9]['open'] / current_open) - 1) * 100
    current_bar['x2_high'] = ((data_5min[index-9]['high'] / current_open) - 1) * 100
    current_bar['x2_low'] = ((data_5min[index-9]['low'] / current_open) - 1) * 100
    current_bar['x2_close'] = ((data_5min[index-9]['close'] / current_open) - 1) * 100
    current_bar['x2_volume'] = ((data_5min[index-9]['volume'] / current_volume) - 1) * 100    
    current_bar['x2_open_real'] = data_5min[index-9]['open'] 
    current_bar['x2_high_real'] = data_5min[index-9]['high'] 
    current_bar['x2_low_real'] = data_5min[index-9]['low'] 
    current_bar['x2_close_real'] = data_5min[index-9]['close']
    current_bar['x2_height'] = current_bar['x2_high'] - current_bar['x2_low']
    current_bar['x2_body'] = ((max([current_bar['x2_open_real'],  current_bar['x2_close_real']]) / min([current_bar['x2_open_real'],  current_bar['x2_close_real']])) - 1) * 100
    current_bar['x2_roc'] = ((current_bar['x2_open_real'] / current_bar['x2_close_real']) - 1) * 100
    current_bar['x2_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema9_close'] = ((current_bar['x2_ema9_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema21_close'] = ((current_bar['x2_ema21_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema55_close'] = ((current_bar['x2_ema55_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema144_close'] = ((current_bar['x2_ema144_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema233_close'] = ((current_bar['x2_ema233_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_x1_close'] = ((current_bar['x2_close_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x2_x1_high'] = ((current_bar['x2_high_real'] / current_bar['x1_high_real']) -1) * 100
    current_bar['x2_x1_low'] = ((current_bar['x2_low_real'] / current_bar['x1_low_real']) -1) * 100    
    
    current_bar['x3_vwap'] = ((data_5min[index-8]['vwap'] / current_open) - 1) * 100
    current_bar['x3_ema9'] = ((data_5min[index-8]['ema9'] / current_open) - 1) * 100
    current_bar['x3_ema21'] = ((data_5min[index-8]['ema21'] / current_open) - 1) * 100
    current_bar['x3_ema55'] = ((data_5min[index-8]['ema55'] / current_open) - 1) * 100
    current_bar['x3_ema144'] = ((data_5min[index-8]['ema144'] / current_open) - 1) * 100
    current_bar['x3_ema233'] = ((data_5min[index-8]['ema233'] / current_open) - 1) * 100        
    current_bar['x3_ema9_real'] = data_5min[index-8]['ema9'] 
    current_bar['x3_ema21_real'] = data_5min[index-8]['ema21'] 
    current_bar['x3_ema55_real'] = data_5min[index-8]['ema55'] 
    current_bar['x3_ema144_real'] = data_5min[index-8]['ema144'] 
    current_bar['x3_ema233_real'] = data_5min[index-8]['ema233'] 
    current_bar['x3_open'] = ((data_5min[index-8]['open'] / current_open) - 1) * 100
    current_bar['x3_high'] = ((data_5min[index-8]['high'] / current_open) - 1) * 100
    current_bar['x3_low'] = ((data_5min[index-8]['low'] / current_open) - 1) * 100
    current_bar['x3_close'] = ((data_5min[index-8]['close'] / current_open) - 1) * 100
    current_bar['x3_volume'] = ((data_5min[index-8]['volume'] / current_volume) - 1) * 100        
    current_bar['x3_open_real'] = data_5min[index-8]['open'] 
    current_bar['x3_high_real'] = data_5min[index-8]['high'] 
    current_bar['x3_low_real'] = data_5min[index-8]['low'] 
    current_bar['x3_close_real'] = data_5min[index-8]['close']
    current_bar['x3_height'] = current_bar['x3_high'] - current_bar['x3_low']
    current_bar['x3_body'] = ((max([current_bar['x3_open_real'],  current_bar['x3_close_real']]) / min([current_bar['x3_open_real'],  current_bar['x3_close_real']])) - 1) * 100
    current_bar['x3_roc'] = ((current_bar['x3_open_real'] / current_bar['x3_close_real']) - 1) * 100
    current_bar['x3_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema9_close'] = ((current_bar['x3_ema9_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema21_close'] = ((current_bar['x3_ema21_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema55_close'] = ((current_bar['x3_ema55_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema144_close'] = ((current_bar['x3_ema144_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema233_close'] = ((current_bar['x3_ema233_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_x2_close'] = ((current_bar['x3_close_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x3_x2_high'] = ((current_bar['x3_high_real'] / current_bar['x2_high_real']) -1) * 100
    current_bar['x3_x2_low'] = ((current_bar['x3_low_real'] / current_bar['x2_low_real']) -1) * 100    
    
    
    current_bar['x4_vwap'] = ((data_5min[index-7]['vwap'] / current_open) - 1) * 100
    current_bar['x4_ema9'] = ((data_5min[index-7]['ema9'] / current_open) - 1) * 100
    current_bar['x4_ema21'] = ((data_5min[index-7]['ema21'] / current_open) - 1) * 100
    current_bar['x4_ema55'] = ((data_5min[index-7]['ema55'] / current_open) - 1) * 100
    current_bar['x4_ema144'] = ((data_5min[index-7]['ema144'] / current_open) - 1) * 100
    current_bar['x4_ema233'] = ((data_5min[index-7]['ema233'] / current_open) - 1) * 100        
    current_bar['x4_ema9_real'] = data_5min[index-7]['ema9'] 
    current_bar['x4_ema21_real'] = data_5min[index-7]['ema21']
    current_bar['x4_ema55_real'] = data_5min[index-7]['ema55']
    current_bar['x4_ema144_real'] = data_5min[index-7]['ema144']
    current_bar['x4_ema233_real'] = data_5min[index-7]['ema233']
    current_bar['x4_open'] = ((data_5min[index-7]['open'] / current_open) - 1) * 100
    current_bar['x4_high'] = ((data_5min[index-7]['high'] / current_open) - 1) * 100
    current_bar['x4_low'] = ((data_5min[index-7]['low'] / current_open) - 1) * 100
    current_bar['x4_close'] = ((data_5min[index-7]['close'] / current_open) - 1) * 100
    current_bar['x4_volume'] = ((data_5min[index-7]['volume'] / current_volume) - 1) * 100            
    current_bar['x4_open_real'] = data_5min[index-7]['open'] 
    current_bar['x4_high_real'] = data_5min[index-7]['high'] 
    current_bar['x4_low_real'] = data_5min[index-7]['low'] 
    current_bar['x4_close_real'] = data_5min[index-7]['close'] 
    current_bar['x4_height'] = current_bar['x4_high'] - current_bar['x4_low']
    current_bar['x4_body'] = ((max([current_bar['x4_open_real'],  current_bar['x4_close_real']]) / min([current_bar['x4_open_real'],  current_bar['x4_close_real']])) - 1) * 100
    current_bar['x4_roc'] = ((current_bar['x4_open_real'] / current_bar['x4_close_real']) - 1) * 100
    current_bar['x4_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema9_close'] = ((current_bar['x4_ema9_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema21_close'] = ((current_bar['x4_ema21_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema55_close'] = ((current_bar['x4_ema55_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema144_close'] = ((current_bar['x4_ema144_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema233_close'] = ((current_bar['x4_ema233_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_x3_close'] = ((current_bar['x4_close_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x4_x3_high'] = ((current_bar['x4_high_real'] / current_bar['x3_high_real']) -1) * 100
    current_bar['x4_x3_low'] = ((current_bar['x4_low_real'] / current_bar['x3_low_real']) -1) * 100        
    
    current_bar['x5_vwap'] = ((data_5min[index-6]['vwap'] / current_open) - 1) * 100
    current_bar['x5_ema9'] = ((data_5min[index-6]['ema9'] / current_open) - 1) * 100
    current_bar['x5_ema21'] = ((data_5min[index-6]['ema21'] / current_open) - 1) * 100
    current_bar['x5_ema55'] = ((data_5min[index-6]['ema55'] / current_open) - 1) * 100
    current_bar['x5_ema144'] = ((data_5min[index-6]['ema144'] / current_open) - 1) * 100
    current_bar['x5_ema233'] = ((data_5min[index-6]['ema233'] / current_open) - 1) * 100        
    current_bar['x5_ema9_real'] = data_5min[index-6]['ema9'] 
    current_bar['x5_ema21_real'] = data_5min[index-6]['ema21'] 
    current_bar['x5_ema55_real'] = data_5min[index-6]['ema55'] 
    current_bar['x5_ema144_real'] = data_5min[index-6]['ema144']
    current_bar['x5_ema233_real'] = data_5min[index-6]['ema233']
    current_bar['x5_open'] = ((data_5min[index-6]['open'] / current_open) - 1) * 100
    current_bar['x5_high'] = ((data_5min[index-6]['high'] / current_open) - 1) * 100
    current_bar['x5_low'] = ((data_5min[index-6]['low'] / current_open) - 1) * 100
    current_bar['x5_close'] = ((data_5min[index-6]['close'] / current_open) - 1) * 100
    current_bar['x5_volume'] = ((data_5min[index-6]['volume'] / current_volume) - 1) * 100            
    current_bar['x5_open_real'] = data_5min[index-6]['open'] 
    current_bar['x5_high_real'] = data_5min[index-6]['high'] 
    current_bar['x5_low_real'] = data_5min[index-6]['low'] 
    current_bar['x5_close_real'] = data_5min[index-6]['close'] 
    current_bar['x5_height'] = current_bar['x5_high'] - current_bar['x5_low']
    current_bar['x5_body'] = ((max([current_bar['x5_open_real'],  current_bar['x5_close_real']]) / min([current_bar['x5_open_real'],  current_bar['x5_close_real']])) - 1) * 100
    current_bar['x5_roc'] = ((current_bar['x5_open_real'] / current_bar['x5_close_real']) - 1) * 100
    current_bar['x5_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema9_close'] = ((current_bar['x5_ema9_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema21_close'] = ((current_bar['x5_ema21_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema55_close'] = ((current_bar['x5_ema55_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema144_close'] = ((current_bar['x5_ema144_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema233_close'] = ((current_bar['x5_ema233_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_x4_close'] = ((current_bar['x5_close_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x5_x4_high'] = ((current_bar['x5_high_real'] / current_bar['x4_high_real']) -1) * 100
    current_bar['x5_x4_low'] = ((current_bar['x5_low_real'] / current_bar['x4_low_real']) -1) * 100        

    current_bar['x6_vwap'] = ((data_5min[index-5]['vwap'] / current_open) - 1) * 100
    current_bar['x6_ema9'] = ((data_5min[index-5]['ema9'] / current_open) - 1) * 100
    current_bar['x6_ema21'] = ((data_5min[index-5]['ema21'] / current_open) - 1) * 100
    current_bar['x6_ema55'] = ((data_5min[index-5]['ema55'] / current_open) - 1) * 100
    current_bar['x6_ema144'] = ((data_5min[index-5]['ema144'] / current_open) - 1) * 100
    current_bar['x6_ema233'] = ((data_5min[index-5]['ema233'] / current_open) - 1) * 100        
    current_bar['x6_ema9_real'] = data_5min[index-5]['ema9'] 
    current_bar['x6_ema21_real'] = data_5min[index-5]['ema21']
    current_bar['x6_ema55_real'] = data_5min[index-5]['ema55']
    current_bar['x6_ema144_real'] = data_5min[index-5]['ema144']
    current_bar['x6_ema233_real'] = data_5min[index-5]['ema233']
    current_bar['x6_open'] = ((data_5min[index-5]['open'] / current_open) - 1) * 100
    current_bar['x6_high'] = ((data_5min[index-5]['high'] / current_open) - 1) * 100
    current_bar['x6_low'] = ((data_5min[index-5]['low'] / current_open) - 1) * 100
    current_bar['x6_close'] = ((data_5min[index-5]['close'] / current_open) - 1) * 100
    current_bar['x6_volume'] = ((data_5min[index-5]['volume'] / current_volume) - 1) * 100            
    current_bar['x6_open_real'] = data_5min[index-5]['open'] 
    current_bar['x6_high_real'] = data_5min[index-5]['high'] 
    current_bar['x6_low_real'] = data_5min[index-5]['low'] 
    current_bar['x6_close_real'] = data_5min[index-5]['close'] 
    current_bar['x6_height'] = current_bar['x6_high'] - current_bar['x6_low']
    current_bar['x6_body'] = ((max([current_bar['x6_open_real'],  current_bar['x6_close_real']]) / min([current_bar['x6_open_real'],  current_bar['x6_close_real']])) - 1) * 100
    current_bar['x6_roc'] = ((current_bar['x6_open_real'] / current_bar['x6_close_real']) - 1) * 100
    current_bar['x6_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema9_close'] = ((current_bar['x6_ema9_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema21_close'] = ((current_bar['x6_ema21_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema55_close'] = ((current_bar['x6_ema55_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema144_close'] = ((current_bar['x6_ema144_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema233_close'] = ((current_bar['x6_ema233_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_x5_close'] = ((current_bar['x6_close_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x6_x5_high'] = ((current_bar['x6_high_real'] / current_bar['x5_high_real']) -1) * 100
    current_bar['x6_x5_low'] = ((current_bar['x6_low_real'] / current_bar['x5_low_real']) -1) * 100        
    
    

    current_bar['x7_vwap'] = ((data_5min[index-4]['vwap'] / current_open) - 1) * 100
    current_bar['x7_ema9'] = ((data_5min[index-4]['ema9'] / current_open) - 1) * 100
    current_bar['x7_ema21'] = ((data_5min[index-4]['ema21'] / current_open) - 1) * 100
    current_bar['x7_ema55'] = ((data_5min[index-4]['ema55'] / current_open) - 1) * 100
    current_bar['x7_ema144'] = ((data_5min[index-4]['ema144'] / current_open) - 1) * 100
    current_bar['x7_ema233'] = ((data_5min[index-4]['ema233'] / current_open) - 1) * 100        
    current_bar['x7_ema9_real'] = data_5min[index-4]['ema9'] 
    current_bar['x7_ema21_real'] = data_5min[index-4]['ema21'] 
    current_bar['x7_ema55_real'] = data_5min[index-4]['ema55'] 
    current_bar['x7_ema144_real'] = data_5min[index-4]['ema144']
    current_bar['x7_ema233_real'] = data_5min[index-4]['ema233']
    current_bar['x7_open'] = ((data_5min[index-4]['open'] / current_open) - 1) * 100
    current_bar['x7_high'] = ((data_5min[index-4]['high'] / current_open) - 1) * 100
    current_bar['x7_low'] = ((data_5min[index-4]['low'] / current_open) - 1) * 100
    current_bar['x7_close'] = ((data_5min[index-4]['close'] / current_open) - 1) * 100
    current_bar['x7_volume'] = ((data_5min[index-4]['volume'] / current_volume) - 1) * 100            
    current_bar['x7_open_real'] = data_5min[index-4]['open'] 
    current_bar['x7_high_real'] = data_5min[index-4]['high'] 
    current_bar['x7_low_real'] = data_5min[index-4]['low'] 
    current_bar['x7_close_real'] = data_5min[index-4]['close'] 
    current_bar['x7_height'] = current_bar['x7_high'] - current_bar['x7_low']
    current_bar['x7_body'] = ((max([current_bar['x7_open_real'],  current_bar['x7_close_real']]) / min([current_bar['x7_open_real'],  current_bar['x7_close_real']])) - 1) * 100
    current_bar['x7_roc'] = ((current_bar['x7_open_real'] / current_bar['x7_close_real']) - 1) * 100
    current_bar['x7_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema9_close'] = ((current_bar['x7_ema9_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema21_close'] = ((current_bar['x7_ema21_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema55_close'] = ((current_bar['x7_ema55_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema144_close'] = ((current_bar['x7_ema144_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema233_close'] = ((current_bar['x7_ema233_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_x6_close'] = ((current_bar['x7_close_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x7_x6_high'] = ((current_bar['x7_high_real'] / current_bar['x6_high_real']) -1) * 100
    current_bar['x7_x6_low'] = ((current_bar['x7_low_real'] / current_bar['x6_low_real']) -1) * 100        
    
    current_bar['x8_vwap'] = ((data_5min[index-3]['vwap'] / current_open) - 1) * 100
    current_bar['x8_ema9'] = ((data_5min[index-3]['ema9'] / current_open) - 1) * 100
    current_bar['x8_ema21'] = ((data_5min[index-3]['ema21'] / current_open) - 1) * 100
    current_bar['x8_ema55'] = ((data_5min[index-3]['ema55'] / current_open) - 1) * 100
    current_bar['x8_ema144'] = ((data_5min[index-3]['ema144'] / current_open) - 1) * 100
    current_bar['x8_ema233'] = ((data_5min[index-3]['ema233'] / current_open) - 1) * 100        
    current_bar['x8_ema9_real'] = data_5min[index-3]['ema9'] 
    current_bar['x8_ema21_real'] = data_5min[index-3]['ema21']
    current_bar['x8_ema55_real'] = data_5min[index-3]['ema55'] 
    current_bar['x8_ema144_real'] = data_5min[index-3]['ema144']
    current_bar['x8_ema233_real'] = data_5min[index-3]['ema233']
    current_bar['x8_open'] = ((data_5min[index-3]['open'] / current_open) - 1) * 100
    current_bar['x8_high'] = ((data_5min[index-3]['high'] / current_open) - 1) * 100
    current_bar['x8_low'] = ((data_5min[index-3]['low'] / current_open) - 1) * 100
    current_bar['x8_close'] = ((data_5min[index-3]['close'] / current_open) - 1) * 100
    current_bar['x8_volume'] = ((data_5min[index-3]['volume'] / current_volume) - 1) * 100            
    current_bar['x8_open_real'] = data_5min[index-3]['open'] 
    current_bar['x8_high_real'] = data_5min[index-3]['high'] 
    current_bar['x8_low_real'] = data_5min[index-3]['low'] 
    current_bar['x8_close_real'] = data_5min[index-3]['close']    
    current_bar['x8_height'] = current_bar['x8_high'] - current_bar['x8_low']
    current_bar['x8_body'] = ((max([current_bar['x8_open_real'],  current_bar['x8_close_real']]) / min([current_bar['x8_open_real'],  current_bar['x8_close_real']])) - 1) * 100
    current_bar['x8_roc'] = ((current_bar['x8_open_real'] / current_bar['x8_close_real']) - 1) * 100
    current_bar['x8_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema9_close'] = ((current_bar['x8_ema9_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema21_close'] = ((current_bar['x8_ema21_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema55_close'] = ((current_bar['x8_ema55_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema144_close'] = ((current_bar['x8_ema144_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema233_close'] = ((current_bar['x8_ema233_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_x7_close'] = ((current_bar['x8_close_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x8_x7_high'] = ((current_bar['x8_high_real'] / current_bar['x7_high_real']) -1) * 100
    current_bar['x8_x7_low'] = ((current_bar['x8_low_real'] / current_bar['x7_low_real']) -1) * 100        
    
    
    current_bar['x9_vwap'] = ((data_5min[index-2]['vwap'] / current_open) - 1) * 100
    current_bar['x9_ema9'] = ((data_5min[index-2]['ema9'] / current_open) - 1) * 100
    current_bar['x9_ema21'] = ((data_5min[index-2]['ema21'] / current_open) - 1) * 100
    current_bar['x9_ema55'] = ((data_5min[index-2]['ema55'] / current_open) - 1) * 100
    current_bar['x9_ema144'] = ((data_5min[index-2]['ema144'] / current_open) - 1) * 100
    current_bar['x9_ema233'] = ((data_5min[index-2]['ema233'] / current_open) - 1) * 100        
    current_bar['x9_ema9_real'] = data_5min[index-2]['ema9'] 
    current_bar['x9_ema21_real'] = data_5min[index-2]['ema21'] 
    current_bar['x9_ema55_real'] = data_5min[index-2]['ema55'] 
    current_bar['x9_ema144_real'] = data_5min[index-2]['ema144']
    current_bar['x9_ema233_real'] = data_5min[index-2]['ema233']
    current_bar['x9_open'] = ((data_5min[index-2]['open'] / current_open) - 1) * 100
    current_bar['x9_high'] = ((data_5min[index-2]['high'] / current_open) - 1) * 100
    current_bar['x9_low'] = ((data_5min[index-2]['low'] / current_open) - 1) * 100
    current_bar['x9_close'] = ((data_5min[index-2]['close'] / current_open) - 1) * 100
    current_bar['x9_volume'] = ((data_5min[index-2]['volume'] / current_volume) - 1) * 100            
    current_bar['x9_open_real'] = data_5min[index-2]['open'] 
    current_bar['x9_high_real'] = data_5min[index-2]['high'] 
    current_bar['x9_low_real'] = data_5min[index-2]['low'] 
    current_bar['x9_close_real'] = data_5min[index-2]['close']
    current_bar['x9_height'] = current_bar['x9_high'] - current_bar['x9_low']
    current_bar['x9_body'] = ((max([current_bar['x9_open_real'],  current_bar['x9_close_real']]) / min([current_bar['x9_open_real'],  current_bar['x9_close_real']])) - 1) * 100
    current_bar['x9_roc'] = ((current_bar['x9_open_real'] / current_bar['x9_close_real']) - 1) * 100
    current_bar['x9_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema9_close'] = ((current_bar['x9_ema9_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema21_close'] = ((current_bar['x9_ema21_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema55_close'] = ((current_bar['x9_ema55_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema144_close'] = ((current_bar['x9_ema144_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema233_close'] = ((current_bar['x9_ema233_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_x8_close'] = ((current_bar['x9_close_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x9_x8_high'] = ((current_bar['x9_high_real'] / current_bar['x8_high_real']) -1) * 100
    current_bar['x9_x8_low'] = ((current_bar['x9_low_real'] / current_bar['x8_low_real']) -1) * 100        
    
    
    current_bar['x10_vwap'] = ((data_5min[index-1]['vwap'] / current_open) - 1) * 100
    current_bar['x10_ema9'] = ((data_5min[index-1]['ema9'] / current_open) - 1) * 100
    current_bar['x10_ema21'] = ((data_5min[index-1]['ema21'] / current_open) - 1) * 100
    current_bar['x10_ema55'] = ((data_5min[index-1]['ema55'] / current_open) - 1) * 100
    current_bar['x10_ema144'] = ((data_5min[index-1]['ema144'] / current_open) - 1) * 100
    current_bar['x10_ema233'] = ((data_5min[index-1]['ema233'] / current_open) - 1) * 100    
    current_bar['x10_ema9_real'] = data_5min[index-1]['ema9'] 
    current_bar['x10_ema21_real'] = data_5min[index-1]['ema21']
    current_bar['x10_ema55_real'] = data_5min[index-1]['ema55']
    current_bar['x10_ema144_real'] = data_5min[index-1]['ema144']
    current_bar['x10_ema233_real'] = data_5min[index-1]['ema233']
    current_bar['x10_open'] = ((data_5min[index-1]['open'] / current_open) - 1) * 100
    current_bar['x10_high'] = ((data_5min[index-1]['high'] / current_open) - 1) * 100
    current_bar['x10_low'] = ((data_5min[index-1]['low'] / current_open) - 1) * 100
    current_bar['x10_close'] = ((data_5min[index-1]['close'] / current_open) - 1) * 100
    current_bar['x10_volume'] = ((data_5min[index-1]['volume'] / current_volume) - 1) * 100            
    current_bar['x10_open_real'] = data_5min[index-1]['open'] 
    current_bar['x10_high_real'] = data_5min[index-1]['high'] 
    current_bar['x10_low_real'] = data_5min[index-1]['low'] 
    current_bar['x10_close_real'] = data_5min[index-1]['close']   
    current_bar['x10_height'] = current_bar['x10_high'] - current_bar['x10_low']
    current_bar['x10_body'] = ((max([current_bar['x10_open_real'],  current_bar['x10_close_real']]) / min([current_bar['x10_open_real'],  current_bar['x10_close_real']])) - 1) * 100
    current_bar['x10_roc'] = ((current_bar['x10_open_real'] / current_bar['x10_close_real']) - 1) * 100
    current_bar['x10_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema9_close'] = ((current_bar['x10_ema9_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema21_close'] = ((current_bar['x10_ema21_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema55_close'] = ((current_bar['x10_ema55_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema144_close'] = ((current_bar['x10_ema144_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema233_close'] = ((current_bar['x10_ema233_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_x9_close'] = ((current_bar['x10_close_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x10_x9_high'] = ((current_bar['x10_high_real'] / current_bar['x9_high_real']) -1) * 100
    current_bar['x10_x9_low'] = ((current_bar['x10_low_real'] / current_bar['x9_low_real']) -1) * 100        
    
    
    current_bar['x11_vwap'] = ((data_5min[index]['vwap'] / current_open) - 1) * 100
    current_bar['x11_ema9'] = ((data_5min[index]['ema9'] / current_open) - 1) * 100
    current_bar['x11_ema21'] = ((data_5min[index]['ema21'] / current_open) - 1) * 100
    current_bar['x11_ema55'] = ((data_5min[index]['ema55'] / current_open) - 1) * 100
    current_bar['x11_ema144'] = ((data_5min[index]['ema144'] / current_open) - 1) * 100
    current_bar['x11_ema233'] = ((data_5min[index]['ema233'] / current_open) - 1) * 100        
    current_bar['x11_ema9_real'] = data_5min[index]['ema9'] 
    current_bar['x11_ema21_real'] = data_5min[index]['ema21'] 
    current_bar['x11_ema55_real'] = data_5min[index]['ema55'] 
    current_bar['x11_ema144_real'] = data_5min[index]['ema144']
    current_bar['x11_ema233_real'] = data_5min[index]['ema233']
    current_bar['x11_open'] = ((data_5min[index]['open'] / current_open) - 1) * 100
    current_bar['x11_high'] = ((data_5min[index]['high'] / current_open) - 1) * 100
    current_bar['x11_low'] = ((data_5min[index]['low'] / current_open) - 1) * 100
    current_bar['x11_close'] = ((data_5min[index]['close'] / current_open) - 1) * 100
    current_bar['x11_volume'] = ((data_5min[index]['volume'] / current_volume) - 1) * 100            
    current_bar['x11_open_real'] = data_5min[index]['open'] 
    current_bar['x11_high_real'] = data_5min[index]['high'] 
    current_bar['x11_low_real'] = data_5min[index]['low'] 
    current_bar['x11_close_real'] = data_5min[index]['close']    
    current_bar['x11_height'] = current_bar['x11_high'] - current_bar['x11_low']
    current_bar['x11_body'] = ((max([current_bar['x11_open_real'],  current_bar['x11_close_real']]) / min([current_bar['x11_open_real'],  current_bar['x11_close_real']])) - 1) * 100
    current_bar['x11_roc'] = ((current_bar['x11_open_real'] / current_bar['x11_close_real']) - 1) * 100
    current_bar['x11_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema9_close'] = ((current_bar['x11_ema9_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema21_close'] = ((current_bar['x11_ema21_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema55_close'] = ((current_bar['x11_ema55_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema144_close'] = ((current_bar['x11_ema144_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema233_close'] = ((current_bar['x11_ema233_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_x10_close'] = ((current_bar['x11_close_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x11_x10_high'] = ((current_bar['x11_high_real'] / current_bar['x10_high_real']) -1) * 100
    current_bar['x11_x10_low'] = ((current_bar['x11_low_real'] / current_bar['x10_low_real']) -1) * 100        

    
    current_bar['x11_high_slope'] = get_slope(current_bar,'high')
    current_bar['x11_low_slope'] = get_slope(current_bar,'low')
    current_bar['x11_close_slope'] = get_slope(current_bar,'low')
    current_bar['x11_volume_slope'] = get_slope(current_bar,'volume')
    
    current_bar['y0_vwap'] = ((data_5min[index+1]['vwap'] / current_open) - 1) * 100
    current_bar['y0_ema9'] = ((data_5min[index+1]['ema9'] / current_open) - 1) * 100
    current_bar['y0_ema21'] = ((data_5min[index+1]['ema21'] / current_open) - 1) * 100
    current_bar['y0_open'] = ((data_5min[index+1]['open'] / current_open) - 1) * 100
    current_bar['y0_high'] = ((data_5min[index+1]['high'] / current_open) - 1) * 100
    current_bar['y0_low'] = ((data_5min[index+1]['low'] / current_open) - 1) * 100
    current_bar['y0_close'] = ((data_5min[index+1]['close'] / current_open) - 1) * 100
    current_bar['y0_volume'] = ((data_5min[index+1]['volume'] / current_volume) - 1) * 100
    current_bar['y0_open_real'] = data_5min[index+1]['open'] 
    current_bar['y0_high_real'] = data_5min[index+1]['high'] 
    current_bar['y0_low_real'] = data_5min[index+1]['low'] 
    current_bar['y0_close_real'] = data_5min[index+1]['close'] 

    current_bar['y1_vwap'] = ((data_5min[index+2]['vwap'] / current_open) - 1) * 100
    current_bar['y1_ema9'] = ((data_5min[index+2]['ema9'] / current_open) - 1) * 100
    current_bar['y1_ema21'] = ((data_5min[index+2]['ema21'] / current_open) - 1) * 100
    current_bar['y1_open'] = ((data_5min[index+2]['open'] / current_open) - 1) * 100
    current_bar['y1_high'] = ((data_5min[index+2]['high'] / current_open) - 1) * 100
    current_bar['y1_low'] = ((data_5min[index+2]['low'] / current_open) - 1) * 100
    current_bar['y1_close'] = ((data_5min[index+2]['close'] / current_open) - 1) * 100
    current_bar['y1_volume'] = ((data_5min[index+2]['volume'] / current_volume) - 1) * 100
    current_bar['y1_open_real'] = data_5min[index+2]['open'] 
    current_bar['y1_high_real'] = data_5min[index+2]['high'] 
    current_bar['y1_low_real'] = data_5min[index+2]['low'] 
    current_bar['y1_close_real'] = data_5min[index+2]['close'] 
    
    current_bar['y2_vwap'] = ((data_5min[index+3]['vwap'] / current_open) - 1) * 100
    current_bar['y2_ema9'] = ((data_5min[index+3]['ema9'] / current_open) - 1) * 100
    current_bar['y2_ema21'] = ((data_5min[index+3]['ema21'] / current_open) - 1) * 100
    current_bar['y2_open'] = ((data_5min[index+3]['open'] / current_open) - 1) * 100
    current_bar['y2_high'] = ((data_5min[index+3]['high'] / current_open) - 1) * 100
    current_bar['y2_low'] = ((data_5min[index+3]['low'] / current_open) - 1) * 100
    current_bar['y2_close'] = ((data_5min[index+3]['close'] / current_open) - 1) * 100
    current_bar['y2_volume'] = ((data_5min[index+3]['volume'] / current_volume) - 1) * 100    
    current_bar['y2_open_real'] = data_5min[index+3]['open'] 
    current_bar['y2_high_real'] = data_5min[index+3]['high'] 
    current_bar['y2_low_real'] = data_5min[index+3]['low'] 
    current_bar['y2_close_real'] = data_5min[index+3]['close'] 

    current_bar['y3_vwap'] = ((data_5min[index-4]['vwap'] / current_open) - 1) * 100
    current_bar['y3_ema9'] = ((data_5min[index+4]['ema9'] / current_open) - 1) * 100
    current_bar['y3_ema21'] = ((data_5min[index+4]['ema21'] / current_open) - 1) * 100
    current_bar['y3_open'] = ((data_5min[index+4]['open'] / current_open) - 1) * 100
    current_bar['y3_high'] = ((data_5min[index+4]['high'] / current_open) - 1) * 100
    current_bar['y3_low'] = ((data_5min[index+4]['low'] / current_open) - 1) * 100
    current_bar['y3_close'] = ((data_5min[index+4]['close'] / current_open) - 1) * 100
    current_bar['y3_volume'] = ((data_5min[index+4]['volume'] / current_volume) - 1) * 100        
    current_bar['y3_open_real'] = data_5min[index+4]['open'] 
    current_bar['y3_high_real'] = data_5min[index+4]['high'] 
    current_bar['y3_low_real'] = data_5min[index+4]['low'] 
    current_bar['y3_close_real'] = data_5min[index+4]['close']

    current_bar['y4_vwap'] = ((data_5min[index+5]['vwap'] / current_open) - 1) * 100
    current_bar['y4_ema9'] = ((data_5min[index+5]['ema9'] / current_open) - 1) * 100
    current_bar['y4_ema21'] = ((data_5min[index+5]['ema21'] / current_open) - 1) * 100
    current_bar['y4_open'] = ((data_5min[index+5]['open'] / current_open) - 1) * 100
    current_bar['y4_high'] = ((data_5min[index+5]['high'] / current_open) - 1) * 100
    current_bar['y4_low'] = ((data_5min[index+5]['low'] / current_open) - 1) * 100
    current_bar['y4_close'] = ((data_5min[index+5]['close'] / current_open) - 1) * 100
    current_bar['y4_volume'] = ((data_5min[index+5]['volume'] / current_volume) - 1) * 100            
    current_bar['y4_open_real'] = data_5min[index+5]['open'] 
    current_bar['y4_high_real'] = data_5min[index+5]['high'] 
    current_bar['y4_low_real'] = data_5min[index+5]['low'] 
    current_bar['y4_close_real'] = data_5min[index+5]['close'] 

    current_bar['y5_vwap'] = ((data_5min[index+6]['vwap'] / current_open) - 1) * 100
    current_bar['y5_ema9'] = ((data_5min[index+6]['ema9'] / current_open) - 1) * 100
    current_bar['y5_ema21'] = ((data_5min[index+6]['ema21'] / current_open) - 1) * 100
    current_bar['y5_open'] = ((data_5min[index+6]['open'] / current_open) - 1) * 100
    current_bar['y5_high'] = ((data_5min[index+6]['high'] / current_open) - 1) * 100
    current_bar['y5_low'] = ((data_5min[index+6]['low'] / current_open) - 1) * 100
    current_bar['y5_close'] = ((data_5min[index+6]['close'] / current_open) - 1) * 100
    current_bar['y5_volume'] = ((data_5min[index+6]['volume'] / current_volume) - 1) * 100                
    current_bar['y5_open_real'] = data_5min[index+6]['open'] 
    current_bar['y5_high_real'] = data_5min[index+6]['high']
    current_bar['y5_low_real'] = data_5min[index+6]['low']
    current_bar['y5_close_real'] = data_5min[index+6]['close']

    current_bar['y6_vwap'] = ((data_5min[index+7]['vwap'] / current_open) - 1) * 100
    current_bar['y6_ema9'] = ((data_5min[index+7]['ema9'] / current_open) - 1) * 100
    current_bar['y6_ema21'] = ((data_5min[index+7]['ema21'] / current_open) - 1) * 100
    current_bar['y6_open'] = ((data_5min[index+7]['open'] / current_open) - 1) * 100
    current_bar['y6_high'] = ((data_5min[index+7]['high'] / current_open) - 1) * 100
    current_bar['y6_low'] = ((data_5min[index+7]['low'] / current_open) - 1) * 100
    current_bar['y6_close'] = ((data_5min[index+7]['close'] / current_open) - 1) * 100
    current_bar['y6_volume'] = ((data_5min[index+7]['volume'] / current_volume) - 1) * 100                
    current_bar['y6_open_real'] = data_5min[index+7]['open'] 
    current_bar['y6_high_real'] = data_5min[index+7]['high']
    current_bar['y6_low_real'] = data_5min[index+7]['low']
    current_bar['y6_close_real'] = data_5min[index+7]['close']

    current_bar['y7_vwap'] = ((data_5min[index+8]['vwap'] / current_open) - 1) * 100
    current_bar['y7_ema9'] = ((data_5min[index+8]['ema9'] / current_open) - 1) * 100
    current_bar['y7_ema21'] = ((data_5min[index+8]['ema21'] / current_open) - 1) * 100
    current_bar['y7_open'] = ((data_5min[index+8]['open'] / current_open) - 1) * 100
    current_bar['y7_high'] = ((data_5min[index+8]['high'] / current_open) - 1) * 100
    current_bar['y7_low'] = ((data_5min[index+8]['low'] / current_open) - 1) * 100
    current_bar['y7_close'] = ((data_5min[index+8]['close'] / current_open) - 1) * 100
    current_bar['y7_volume'] = ((data_5min[index+8]['volume'] / current_volume) - 1) * 100                
    current_bar['y7_open_real'] = data_5min[index+8]['open'] 
    current_bar['y7_high_real'] = data_5min[index+8]['high']
    current_bar['y7_low_real'] = data_5min[index+8]['low']
    current_bar['y7_close_real'] = data_5min[index+8]['close']

    current_bar['y8_vwap'] = ((data_5min[index+9]['vwap'] / current_open) - 1) * 100
    current_bar['y8_ema9'] = ((data_5min[index+9]['ema9'] / current_open) - 1) * 100
    current_bar['y8_ema21'] = ((data_5min[index+9]['ema21'] / current_open) - 1) * 100
    current_bar['y8_open'] = ((data_5min[index+9]['open'] / current_open) - 1) * 100
    current_bar['y8_high'] = ((data_5min[index+9]['high'] / current_open) - 1) * 100
    current_bar['y8_low'] = ((data_5min[index+9]['low'] / current_open) - 1) * 100
    current_bar['y8_close'] = ((data_5min[index+9]['close'] / current_open) - 1) * 100
    current_bar['y8_volume'] = ((data_5min[index+9]['volume'] / current_volume) - 1) * 100                
    current_bar['y8_open_real'] = data_5min[index+9]['open'] 
    current_bar['y8_high_real'] = data_5min[index+9]['high']
    current_bar['y8_low_real'] = data_5min[index+9]['low']
    current_bar['y8_close_real'] = data_5min[index+9]['close']
    
    current_bar['y9_vwap'] = ((data_5min[index+10]['vwap'] / current_open) - 1) * 100
    current_bar['y9_ema9'] = ((data_5min[index+10]['ema9'] / current_open) - 1) * 100
    current_bar['y9_ema21'] = ((data_5min[index+10]['ema21'] / current_open) - 1) * 100
    current_bar['y9_open'] = ((data_5min[index+10]['open'] / current_open) - 1) * 100
    current_bar['y9_high'] = ((data_5min[index+10]['high'] / current_open) - 1) * 100
    current_bar['y9_low'] = ((data_5min[index+10]['low'] / current_open) - 1) * 100
    current_bar['y9_close'] = ((data_5min[index+10]['close'] / current_open) - 1) * 100
    current_bar['y9_volume'] = ((data_5min[index+10]['volume'] / current_volume) - 1) * 100                
    current_bar['y9_open_real'] = data_5min[index+10]['open'] 
    current_bar['y9_high_real'] = data_5min[index+10]['high']
    current_bar['y9_low_real'] = data_5min[index+10]['low']
    current_bar['y9_close_real'] = data_5min[index+10]['close']

    current_bar['y10_vwap'] = ((data_5min[index+11]['vwap'] / current_open) - 1) * 100
    current_bar['y10_ema9'] = ((data_5min[index+11]['ema9'] / current_open) - 1) * 100
    current_bar['y10_ema21'] = ((data_5min[index+11]['ema21'] / current_open) - 1) * 100
    current_bar['y10_open'] = ((data_5min[index+11]['open'] / current_open) - 1) * 100
    current_bar['y10_high'] = ((data_5min[index+11]['high'] / current_open) - 1) * 100
    current_bar['y10_low'] = ((data_5min[index+11]['low'] / current_open) - 1) * 100
    current_bar['y10_close'] = ((data_5min[index+11]['close'] / current_open) - 1) * 100
    current_bar['y10_volume'] = ((data_5min[index+11]['volume'] / current_volume) - 1) * 100                
    current_bar['y10_open_real'] = data_5min[index+11]['open'] 
    current_bar['y10_high_real'] = data_5min[index+11]['high']
    current_bar['y10_low_real'] = data_5min[index+11]['low']
    current_bar['y10_close_real'] = data_5min[index+11]['close']    
    
    current_bar['y11_vwap'] = ((data_5min[index+12]['vwap'] / current_open) - 1) * 100
    current_bar['y11_ema9'] = ((data_5min[index+12]['ema9'] / current_open) - 1) * 100
    current_bar['y11_ema21'] = ((data_5min[index+12]['ema21'] / current_open) - 1) * 100
    current_bar['y11_open'] = ((data_5min[index+12]['open'] / current_open) - 1) * 100
    current_bar['y11_high'] = ((data_5min[index+12]['high'] / current_open) - 1) * 100
    current_bar['y11_low'] = ((data_5min[index+12]['low'] / current_open) - 1) * 100
    current_bar['y11_close'] = ((data_5min[index+12]['close'] / current_open) - 1) * 100
    current_bar['y11_volume'] = ((data_5min[index+12]['volume'] / current_volume) - 1) * 100                
    current_bar['y11_open_real'] = data_5min[index+12]['open'] 
    current_bar['y11_high_real'] = data_5min[index+12]['high']
    current_bar['y11_low_real'] = data_5min[index+12]['low']
    current_bar['y11_close_real'] = data_5min[index+12]['close']        

    current_bar['y12_vwap'] = ((data_5min[index+13]['vwap'] / current_open) - 1) * 100
    current_bar['y12_ema9'] = ((data_5min[index+13]['ema9'] / current_open) - 1) * 100
    current_bar['y12_ema21'] = ((data_5min[index+13]['ema21'] / current_open) - 1) * 100
    current_bar['y12_open'] = ((data_5min[index+13]['open'] / current_open) - 1) * 100
    current_bar['y12_high'] = ((data_5min[index+13]['high'] / current_open) - 1) * 100
    current_bar['y12_low'] = ((data_5min[index+13]['low'] / current_open) - 1) * 100
    current_bar['y12_close'] = ((data_5min[index+13]['close'] / current_open) - 1) * 100
    current_bar['y12_volume'] = ((data_5min[index+13]['volume'] / current_volume) - 1) * 100                
    current_bar['y12_open_real'] = data_5min[index+13]['open'] 
    current_bar['y12_high_real'] = data_5min[index+13]['high']
    current_bar['y12_low_real'] = data_5min[index+13]['low']
    current_bar['y12_close_real'] = data_5min[index+13]['close']  

    current_bar['y13_vwap'] = ((data_5min[index+14]['vwap'] / current_open) - 1) * 100
    current_bar['y13_ema9'] = ((data_5min[index+14]['ema9'] / current_open) - 1) * 100
    current_bar['y13_ema21'] = ((data_5min[index+14]['ema21'] / current_open) - 1) * 100
    current_bar['y13_open'] = ((data_5min[index+14]['open'] / current_open) - 1) * 100
    current_bar['y13_high'] = ((data_5min[index+14]['high'] / current_open) - 1) * 100
    current_bar['y13_low'] = ((data_5min[index+14]['low'] / current_open) - 1) * 100
    current_bar['y13_close'] = ((data_5min[index+14]['close'] / current_open) - 1) * 100
    current_bar['y13_volume'] = ((data_5min[index+14]['volume'] / current_volume) - 1) * 100                
    current_bar['y13_open_real'] = data_5min[index+14]['open'] 
    current_bar['y13_high_real'] = data_5min[index+14]['high']
    current_bar['y13_low_real'] = data_5min[index+14]['low']
    current_bar['y13_close_real'] = data_5min[index+14]['close']  

    current_bar['y14_vwap'] = ((data_5min[index+15]['vwap'] / current_open) - 1) * 100
    current_bar['y14_ema9'] = ((data_5min[index+15]['ema9'] / current_open) - 1) * 100
    current_bar['y14_ema21'] = ((data_5min[index+15]['ema21'] / current_open) - 1) * 100
    current_bar['y14_open'] = ((data_5min[index+15]['open'] / current_open) - 1) * 100
    current_bar['y14_high'] = ((data_5min[index+15]['high'] / current_open) - 1) * 100
    current_bar['y14_low'] = ((data_5min[index+15]['low'] / current_open) - 1) * 100
    current_bar['y14_close'] = ((data_5min[index+15]['close'] / current_open) - 1) * 100
    current_bar['y14_volume'] = ((data_5min[index+15]['volume'] / current_volume) - 1) * 100                
    current_bar['y14_open_real'] = data_5min[index+15]['open'] 
    current_bar['y14_high_real'] = data_5min[index+15]['high']
    current_bar['y14_low_real'] = data_5min[index+15]['low']
    current_bar['y14_close_real'] = data_5min[index+15]['close']  

    current_bar['y15_vwap'] = ((data_5min[index+16]['vwap'] / current_open) - 1) * 100
    current_bar['y15_ema9'] = ((data_5min[index+16]['ema9'] / current_open) - 1) * 100
    current_bar['y15_ema21'] = ((data_5min[index+16]['ema21'] / current_open) - 1) * 100
    current_bar['y15_open'] = ((data_5min[index+16]['open'] / current_open) - 1) * 100
    current_bar['y15_high'] = ((data_5min[index+16]['high'] / current_open) - 1) * 100
    current_bar['y15_low'] = ((data_5min[index+16]['low'] / current_open) - 1) * 100
    current_bar['y15_close'] = ((data_5min[index+16]['close'] / current_open) - 1) * 100
    current_bar['y15_volume'] = ((data_5min[index+16]['volume'] / current_volume) - 1) * 100                
    current_bar['y15_open_real'] = data_5min[index+16]['open'] 
    current_bar['y15_high_real'] = data_5min[index+16]['high']
    current_bar['y15_low_real'] = data_5min[index+16]['low']
    current_bar['y15_close_real'] = data_5min[index+16]['close']  

    current_bar['y16_vwap'] = ((data_5min[index+17]['vwap'] / current_open) - 1) * 100
    current_bar['y16_ema9'] = ((data_5min[index+17]['ema9'] / current_open) - 1) * 100
    current_bar['y16_ema21'] = ((data_5min[index+17]['ema21'] / current_open) - 1) * 100
    current_bar['y16_open'] = ((data_5min[index+17]['open'] / current_open) - 1) * 100
    current_bar['y16_high'] = ((data_5min[index+17]['high'] / current_open) - 1) * 100
    current_bar['y16_low'] = ((data_5min[index+17]['low'] / current_open) - 1) * 100
    current_bar['y16_close'] = ((data_5min[index+17]['close'] / current_open) - 1) * 100
    current_bar['y16_volume'] = ((data_5min[index+17]['volume'] / current_volume) - 1) * 100                
    current_bar['y16_open_real'] = data_5min[index+17]['open'] 
    current_bar['y16_high_real'] = data_5min[index+17]['high']
    current_bar['y16_low_real'] = data_5min[index+17]['low']
    current_bar['y16_close_real'] = data_5min[index+17]['close']  

    current_bar['y17_vwap'] = ((data_5min[index-18]['vwap'] / current_open) - 1) * 100
    current_bar['y17_ema9'] = ((data_5min[index+18]['ema9'] / current_open) - 1) * 100
    current_bar['y17_ema21'] = ((data_5min[index+18]['ema21'] / current_open) - 1) * 100
    current_bar['y17_open'] = ((data_5min[index+18]['open'] / current_open) - 1) * 100
    current_bar['y17_high'] = ((data_5min[index+18]['high'] / current_open) - 1) * 100
    current_bar['y17_low'] = ((data_5min[index+18]['low'] / current_open) - 1) * 100
    current_bar['y17_close'] = ((data_5min[index+18]['close'] / current_open) - 1) * 100
    current_bar['y17_volume'] = ((data_5min[index+18]['volume'] / current_volume) - 1) * 100                
    current_bar['y17_open_real'] = data_5min[index+18]['open'] 
    current_bar['y17_high_real'] = data_5min[index+18]['high']
    current_bar['y17_low_real'] = data_5min[index+18]['low']
    current_bar['y17_close_real'] = data_5min[index+18]['close']  

    current_bar['y18_vwap'] = ((data_5min[index-19]['vwap'] / current_open) - 1) * 100
    current_bar['y18_ema9'] = ((data_5min[index+19]['ema9'] / current_open) - 1) * 100
    current_bar['y18_ema21'] = ((data_5min[index+19]['ema21'] / current_open) - 1) * 100
    current_bar['y18_open'] = ((data_5min[index+19]['open'] / current_open) - 1) * 100
    current_bar['y18_high'] = ((data_5min[index+19]['high'] / current_open) - 1) * 100
    current_bar['y18_low'] = ((data_5min[index+19]['low'] / current_open) - 1) * 100
    current_bar['y18_close'] = ((data_5min[index+19]['close'] / current_open) - 1) * 100
    current_bar['y18_volume'] = ((data_5min[index+19]['volume'] / current_volume) - 1) * 100                
    current_bar['y18_open_real'] = data_5min[index+19]['open'] 
    current_bar['y18_high_real'] = data_5min[index+19]['high']
    current_bar['y18_low_real'] = data_5min[index+19]['low']
    current_bar['y18_close_real'] = data_5min[index+19]['close']  

    current_bar['y19_vwap'] = ((data_5min[index+20]['vwap'] / current_open) - 1) * 100
    current_bar['y19_ema9'] = ((data_5min[index+20]['ema9'] / current_open) - 1) * 100
    current_bar['y19_ema21'] = ((data_5min[index+20]['ema21'] / current_open) - 1) * 100
    current_bar['y19_open'] = ((data_5min[index+20]['open'] / current_open) - 1) * 100
    current_bar['y19_high'] = ((data_5min[index+20]['high'] / current_open) - 1) * 100
    current_bar['y19_low'] = ((data_5min[index+20]['low'] / current_open) - 1) * 100
    current_bar['y19_close'] = ((data_5min[index+20]['close'] / current_open) - 1) * 100
    current_bar['y19_volume'] = ((data_5min[index+20]['volume'] / current_volume) - 1) * 100                
    current_bar['y19_open_real'] = data_5min[index+20]['open'] 
    current_bar['y19_high_real'] = data_5min[index+20]['high']
    current_bar['y19_low_real'] = data_5min[index+20]['low']
    current_bar['y19_close_real'] = data_5min[index+20]['close']  

    current_bar['y20_vwap'] = ((data_5min[index+21]['vwap'] / current_open) - 1) * 100
    current_bar['y20_ema9'] = ((data_5min[index+21]['ema9'] / current_open) - 1) * 100
    current_bar['y20_ema21'] = ((data_5min[index+21]['ema21'] / current_open) - 1) * 100
    current_bar['y20_open'] = ((data_5min[index+21]['open'] / current_open) - 1) * 100
    current_bar['y20_high'] = ((data_5min[index+21]['high'] / current_open) - 1) * 100
    current_bar['y20_low'] = ((data_5min[index+21]['low'] / current_open) - 1) * 100
    current_bar['y20_close'] = ((data_5min[index+21]['close'] / current_open) - 1) * 100
    current_bar['y20_volume'] = ((data_5min[index+21]['volume'] / current_volume) - 1) * 100                
    current_bar['y20_open_real'] = data_5min[index+21]['open'] 
    current_bar['y20_high_real'] = data_5min[index+21]['high']
    current_bar['y20_low_real'] = data_5min[index+21]['low']
    current_bar['y20_close_real'] = data_5min[index+21]['close']  

    current_bar['y21_vwap'] = ((data_5min[index+22]['vwap'] / current_open) - 1) * 100
    current_bar['y21_ema9'] = ((data_5min[index+22]['ema9'] / current_open) - 1) * 100
    current_bar['y21_ema21'] = ((data_5min[index+22]['ema21'] / current_open) - 1) * 100
    current_bar['y21_open'] = ((data_5min[index+22]['open'] / current_open) - 1) * 100
    current_bar['y21_high'] = ((data_5min[index+22]['high'] / current_open) - 1) * 100
    current_bar['y21_low'] = ((data_5min[index+22]['low'] / current_open) - 1) * 100
    current_bar['y21_close'] = ((data_5min[index+22]['close'] / current_open) - 1) * 100
    current_bar['y21_volume'] = ((data_5min[index+22]['volume'] / current_volume) - 1) * 100                
    current_bar['y21_open_real'] = data_5min[index+22]['open'] 
    current_bar['y21_high_real'] = data_5min[index+22]['high']
    current_bar['y21_low_real'] = data_5min[index+22]['low']
    current_bar['y21_close_real'] = data_5min[index+22]['close']  

    current_bar['y22_vwap'] = ((data_5min[index+23]['vwap'] / current_open) - 1) * 100
    current_bar['y22_ema9'] = ((data_5min[index+23]['ema9'] / current_open) - 1) * 100
    current_bar['y22_ema21'] = ((data_5min[index+23]['ema21'] / current_open) - 1) * 100
    current_bar['y22_open'] = ((data_5min[index+23]['open'] / current_open) - 1) * 100
    current_bar['y22_high'] = ((data_5min[index+23]['high'] / current_open) - 1) * 100
    current_bar['y22_low'] = ((data_5min[index+23]['low'] / current_open) - 1) * 100
    current_bar['y22_close'] = ((data_5min[index+23]['close'] / current_open) - 1) * 100
    current_bar['y22_volume'] = ((data_5min[index+23]['volume'] / current_volume) - 1) * 100                
    current_bar['y22_open_real'] = data_5min[index+23]['open'] 
    current_bar['y22_high_real'] = data_5min[index+23]['high']
    current_bar['y22_low_real'] = data_5min[index+23]['low']
    current_bar['y22_close_real'] = data_5min[index+23]['close']  

    current_bar['y23_vwap'] = ((data_5min[index+24]['vwap'] / current_open) - 1) * 100
    current_bar['y23_ema9'] = ((data_5min[index+24]['ema9'] / current_open) - 1) * 100
    current_bar['y23_ema21'] = ((data_5min[index+24]['ema21'] / current_open) - 1) * 100
    current_bar['y23_open'] = ((data_5min[index+24]['open'] / current_open) - 1) * 100
    current_bar['y23_high'] = ((data_5min[index+24]['high'] / current_open) - 1) * 100
    current_bar['y23_low'] = ((data_5min[index+24]['low'] / current_open) - 1) * 100
    current_bar['y23_close'] = ((data_5min[index+24]['close'] / current_open) - 1) * 100
    current_bar['y23_volume'] = ((data_5min[index+24]['volume'] / current_volume) - 1) * 100                
    current_bar['y23_open_real'] = data_5min[index+24]['open'] 
    current_bar['y23_high_real'] = data_5min[index+24]['high']
    current_bar['y23_low_real'] = data_5min[index+24]['low']
    current_bar['y23_close_real'] = data_5min[index+24]['close']  
    
    
    current_bar['chart_data'] = [
        {
        'date': 0,
        'open': current_bar['x0_open'],
        'high': current_bar['x0_high'],
        'low': current_bar['x0_low'],
        'close': current_bar['x0_close'],
        'ema9': current_bar['x0_ema9'],
        'vwap': current_bar['x0_vwap'],
        'ema21': current_bar['x0_ema21'],
        'open_real': current_bar['x0_open_real'],
        'high_real': current_bar['x0_high_real'],
        'low_real': current_bar['x0_low_real'],
        'close_real': current_bar['x0_close_real']
        },
        {
        'date': 1,
        'open': current_bar['x1_open'],
        'high': current_bar['x1_high'],
        'low': current_bar['x1_low'],
        'close': current_bar['x1_close'],
        'ema9': current_bar['x1_ema9'],
        'vwap': current_bar['x1_vwap'],
        'ema21': current_bar['x1_ema21'],
        'open_real': current_bar['x1_open_real'],
        'high_real': current_bar['x1_high_real'],
        'low_real': current_bar['x1_low_real'],
        'close_real': current_bar['x1_close_real']
        },
        {
        'date': 2,
        'open': current_bar['x2_open'],
        'high': current_bar['x2_high'],
        'low': current_bar['x2_low'],
        'close': current_bar['x2_close'],
        'ema9': current_bar['x2_ema9'],
        'vwap': current_bar['x2_vwap'],
        'ema21': current_bar['x2_ema21'],
        'open_real': current_bar['x2_open_real'],
        'high_real': current_bar['x2_high_real'],
        'low_real': current_bar['x2_low_real'],
        'close_real': current_bar['x2_close_real']
        },
        {
        'date': 3,
        'open': current_bar['x3_open'],
        'high': current_bar['x3_high'],
        'low': current_bar['x3_low'],
        'close': current_bar['x3_close'],
        'ema9': current_bar['x3_ema9'],
        'vwap': current_bar['x3_vwap'],
        'ema21': current_bar['x3_ema21'],
        'open_real': current_bar['x3_open_real'],
        'high_real': current_bar['x3_high_real'],
        'low_real': current_bar['x3_low_real'],
        'close_real': current_bar['x3_close_real']
        },     
        {
        'date': 4,
        'open': current_bar['x4_open'],
        'high': current_bar['x4_high'],
        'low': current_bar['x4_low'],
        'close': current_bar['x4_close'],
        'ema9': current_bar['x4_ema9'],
        'vwap': current_bar['x4_vwap'],
        'ema21': current_bar['x4_ema21'],
        'open_real': current_bar['x4_open_real'],
        'high_real': current_bar['x4_high_real'],
        'low_real': current_bar['x4_low_real'],
        'close_real': current_bar['x4_close_real']
        },             
        {
        'date': 5,
        'open': current_bar['x5_open'],
        'high': current_bar['x5_high'],
        'low': current_bar['x5_low'],
        'close': current_bar['x5_close'],
        'ema9': current_bar['x5_ema9'],
        'vwap': current_bar['x5_vwap'],
        'ema21': current_bar['x5_ema21'],
        'open_real': current_bar['x5_open_real'],
        'high_real': current_bar['x5_high_real'],
        'low_real': current_bar['x5_low_real'],
        'close_real': current_bar['x5_close_real']
        },
        {
        'date': 6,
        'open': current_bar['x6_open'],
        'high': current_bar['x6_high'],
        'low': current_bar['x6_low'],
        'close': current_bar['x6_close'],
        'ema9': current_bar['x6_ema9'],
        'vwap': current_bar['x6_vwap'],
        'ema21': current_bar['x6_ema21'],
        'open_real': current_bar['x6_open_real'],
        'high_real': current_bar['x6_high_real'],
        'low_real': current_bar['x6_low_real'],
        'close_real': current_bar['x6_close_real']
        },
        {
        'date': 7,
        'open': current_bar['x7_open'],
        'high': current_bar['x7_high'],
        'low': current_bar['x7_low'],
        'close': current_bar['x7_close'],
        'ema9': current_bar['x7_ema9'],
        'vwap': current_bar['x7_vwap'],
        'ema21': current_bar['x7_ema21'],
        'open_real': current_bar['x7_open_real'],
        'high_real': current_bar['x7_high_real'],
        'low_real': current_bar['x7_low_real'],
        'close_real': current_bar['x7_close_real']
        },
        {
        'date': 8,
        'open': current_bar['x8_open'],
        'high': current_bar['x8_high'],
        'low': current_bar['x8_low'],
        'close': current_bar['x8_close'],
        'ema9': current_bar['x8_ema9'],
        'vwap': current_bar['x8_vwap'],
        'ema21': current_bar['x8_ema21'],
        'open_real': current_bar['x8_open_real'],
        'high_real': current_bar['x8_high_real'],
        'low_real': current_bar['x8_low_real'],
        'close_real': current_bar['x8_close_real']
        },
        {
        'date': 9,
        'open': current_bar['x9_open'],
        'high': current_bar['x9_high'],
        'low': current_bar['x9_low'],
        'close': current_bar['x9_close'],
        'ema9': current_bar['x9_ema9'],
        'vwap': current_bar['x9_vwap'],
        'ema21': current_bar['x9_ema21'],
        'open_real': current_bar['x9_open_real'],
        'high_real': current_bar['x9_high_real'],
        'low_real': current_bar['x9_low_real'],
        'close_real': current_bar['x9_close_real']
        },        
        {
        'date': 10,
        'open': current_bar['x10_open'],
        'high': current_bar['x10_high'],
        'low': current_bar['x10_low'],
        'close': current_bar['x10_close'],
        'ema9': current_bar['x10_ema9'],
        'vwap': current_bar['x10_vwap'],
        'ema21': current_bar['x10_ema21'],
        'open_real': current_bar['x10_open_real'],
        'high_real': current_bar['x10_high_real'],
        'low_real': current_bar['x10_low_real'],
        'close_real': current_bar['x10_close_real']
        },        
        {
        'date': 11,
        'open': current_bar['x11_open'],
        'high': current_bar['x11_high'],
        'low': current_bar['x11_low'],
        'close': current_bar['x11_close'],
        'ema9': current_bar['x11_ema9'],
        'vwap': current_bar['x11_vwap'],
        'ema21': current_bar['x11_ema21'],
        'open_real': current_bar['x11_open_real'],
        'high_real': current_bar['x11_high_real'],
        'low_real': current_bar['x11_low_real'],
        'close_real': current_bar['x11_close_real']
        },                
        {
        'date': 12,
        'open': current_bar['y0_open'],
        'high': current_bar['y0_high'],
        'low': current_bar['y0_low'],
        'close': current_bar['y0_close'],
        'ema9': current_bar['y0_ema9'],
        'vwap': current_bar['y0_vwap'],
        'ema21': current_bar['y0_ema21'],
        'open_real': current_bar['y0_open_real'],
        'high_real': current_bar['y0_high_real'],
        'low_real': current_bar['y0_low_real'],
        'close_real': current_bar['y0_close_real']
        },
        {
        'date': 13,
        'open': current_bar['y1_open'],
        'high': current_bar['y1_high'],
        'low': current_bar['y1_low'],
        'close': current_bar['y1_close'],
        'ema9': current_bar['y1_ema9'],
        'vwap': current_bar['y1_vwap'],
        'ema21': current_bar['y1_ema21'],           
        'open_real': current_bar['y1_open_real'],
        'high_real': current_bar['y1_high_real'],
        'low_real': current_bar['y1_low_real'],
        'close_real': current_bar['y1_close_real']
        },
        {
        'date': 14,
        'open': current_bar['y2_open'],
        'high': current_bar['y2_high'],
        'low': current_bar['y2_low'],
        'close': current_bar['y2_close'],
        'ema9': current_bar['y2_ema9'],
        'vwap': current_bar['y2_vwap'],
        'ema21': current_bar['y2_ema21'],
        'open_real': current_bar['y2_open_real'],
        'high_real': current_bar['y2_high_real'],
        'low_real': current_bar['y2_low_real'],
        'close_real': current_bar['y2_close_real']
        },
        {
        'date': 15,
        'open': current_bar['y3_open'],
        'high': current_bar['y3_high'],
        'low': current_bar['y3_low'],
        'close': current_bar['y3_close'],
        'ema9': current_bar['y3_ema9'],
        'vwap': current_bar['y3_vwap'],
        'ema21': current_bar['y3_ema21'],
        'open_real': current_bar['y3_open_real'],
        'high_real': current_bar['y3_high_real'],
        'low_real': current_bar['y3_low_real'],
        'close_real': current_bar['y3_close_real']
        },     
        {
        'date': 16,
        'open': current_bar['y4_open'],
        'high': current_bar['y4_high'],
        'low': current_bar['y4_low'],
        'close': current_bar['y4_close'],
        'ema9': current_bar['y4_ema9'],
        'vwap': current_bar['y4_vwap'],
        'ema21': current_bar['y4_ema21'],
        'open_real': current_bar['y4_open_real'],
        'high_real': current_bar['y4_high_real'],
        'low_real': current_bar['y4_low_real'],
        'close_real': current_bar['y4_close_real']
        },    
        {
        'date': 17,
        'open': current_bar['y5_open'],
        'high': current_bar['y5_high'],
        'low': current_bar['y5_low'],
        'close': current_bar['y5_close'],
        'ema9': current_bar['y5_ema9'],
        'vwap': current_bar['y5_vwap'],
        'ema21': current_bar['y5_ema21'],
        'open_real': current_bar['y5_open_real'],
        'high_real': current_bar['y5_high_real'],
        'low_real': current_bar['y5_low_real'],
        'close_real': current_bar['y5_close_real']
        },    
        {
        'date': 18,
        'open': current_bar['y6_open'],
        'high': current_bar['y6_high'],
        'low': current_bar['y6_low'],
        'close': current_bar['y6_close'],
        'ema9': current_bar['y6_ema9'],
        'vwap': current_bar['y6_vwap'],
        'ema21': current_bar['y6_ema21'],
        'open_real': current_bar['y6_open_real'],
        'high_real': current_bar['y6_high_real'],
        'low_real': current_bar['y6_low_real'],
        'close_real': current_bar['y6_close_real']
        },    
        {
        'date': 19,
        'open': current_bar['y7_open'],
        'high': current_bar['y7_high'],
        'low': current_bar['y7_low'],
        'close': current_bar['y7_close'],
        'ema9': current_bar['y7_ema9'],
        'vwap': current_bar['y7_vwap'],
        'ema21': current_bar['y7_ema21'],
        'open_real': current_bar['y7_open_real'],
        'high_real': current_bar['y7_high_real'],
        'low_real': current_bar['y7_low_real'],
        'close_real': current_bar['y7_close_real']
        },    
        {
        'date': 20,
        'open': current_bar['y8_open'],
        'high': current_bar['y8_high'],
        'low': current_bar['y8_low'],
        'close': current_bar['y8_close'],
        'ema9': current_bar['y8_ema9'],
        'vwap': current_bar['y8_vwap'],
        'ema21': current_bar['y8_ema21'],
        'open_real': current_bar['y8_open_real'],
        'high_real': current_bar['y8_high_real'],
        'low_real': current_bar['y8_low_real'],
        'close_real': current_bar['y8_close_real']
        },         
        {
        'date': 21,
        'open': current_bar['y9_open'],
        'high': current_bar['y9_high'],
        'low': current_bar['y9_low'],
        'close': current_bar['y9_close'],
        'ema9': current_bar['y9_ema9'],
        'vwap': current_bar['y9_vwap'],
        'ema21': current_bar['y9_ema21'],
        'open_real': current_bar['y9_open_real'],
        'high_real': current_bar['y9_high_real'],
        'low_real': current_bar['y9_low_real'],
        'close_real': current_bar['y9_close_real']
        },         
        {
        'date': 22,
        'open': current_bar['y10_open'],
        'high': current_bar['y10_high'],
        'low': current_bar['y10_low'],
        'close': current_bar['y10_close'],
        'ema9': current_bar['y10_ema9'],
        'vwap': current_bar['y10_vwap'],
        'ema21': current_bar['y10_ema21'],
        'open_real': current_bar['y10_open_real'],
        'high_real': current_bar['y10_high_real'],
        'low_real': current_bar['y10_low_real'],
        'close_real': current_bar['y10_close_real']
        },                 
        {
        'date': 23,
        'open': current_bar['y11_open'],
        'high': current_bar['y11_high'],
        'low': current_bar['y11_low'],
        'close': current_bar['y11_close'],
        'ema9': current_bar['y11_ema9'],
        'vwap': current_bar['y11_vwap'],
        'ema21': current_bar['y11_ema21'], 
        'open_real': current_bar['y11_open_real'],
        'high_real': current_bar['y11_high_real'],
        'low_real': current_bar['y1_low_real'],
        'close_real': current_bar['y11_close_real']
        },
        {
        'date': 24,
        'open': current_bar['y12_open'],
        'high': current_bar['y12_high'],
        'low': current_bar['y12_low'],
        'close': current_bar['y12_close'],
        'ema9': current_bar['y12_ema9'],
        'vwap': current_bar['y12_vwap'],
        'ema21': current_bar['y12_ema21'], 
        'open_real': current_bar['y12_open_real'],
        'high_real': current_bar['y12_high_real'],
        'low_real': current_bar['y12_low_real'],
        'close_real': current_bar['y12_close_real']
        },

        {
        'date': 25,
        'open': current_bar['y13_open'],
        'high': current_bar['y13_high'],
        'low': current_bar['y13_low'],
        'close': current_bar['y13_close'],
        'ema9': current_bar['y13_ema9'],
        'vwap': current_bar['y13_vwap'],
        'ema21': current_bar['y13_ema21'], 
        'open_real': current_bar['y13_open_real'],
        'high_real': current_bar['y13_high_real'],
        'low_real': current_bar['y13_low_real'],
        'close_real': current_bar['y13_close_real']
        },

        {
        'date': 26,
        'open': current_bar['y14_open'],
        'high': current_bar['y14_high'],
        'low': current_bar['y14_low'],
        'close': current_bar['y14_close'],
        'ema9': current_bar['y14_ema9'],
        'vwap': current_bar['y14_vwap'],
        'ema21': current_bar['y14_ema21'], 
        'open_real': current_bar['y14_open_real'],
        'high_real': current_bar['y14_high_real'],
        'low_real': current_bar['y14_low_real'],
        'close_real': current_bar['y14_close_real']
        },

        {
        'date': 27,
        'open': current_bar['y15_open'],
        'high': current_bar['y15_high'],
        'low': current_bar['y15_low'],
        'close': current_bar['y15_close'],
        'ema9': current_bar['y15_ema9'],
        'vwap': current_bar['y15_vwap'],
        'ema21': current_bar['y15_ema21'], 
        'open_real': current_bar['y15_open_real'],
        'high_real': current_bar['y15_high_real'],
        'low_real': current_bar['y15_low_real'],
        'close_real': current_bar['y15_close_real']
        },

        {
        'date': 28,
        'open': current_bar['y16_open'],
        'high': current_bar['y16_high'],
        'low': current_bar['y16_low'],
        'close': current_bar['y16_close'],
        'ema9': current_bar['y16_ema9'],
        'vwap': current_bar['y16_vwap'],
        'ema21': current_bar['y16_ema21'], 
        'open_real': current_bar['y16_open_real'],
        'high_real': current_bar['y16_high_real'],
        'low_real': current_bar['y16_low_real'],
        'close_real': current_bar['y16_close_real']
        },
        
        
        {
        'date': 29,
        'open': current_bar['y17_open'],
        'high': current_bar['y17_high'],
        'low': current_bar['y17_low'],
        'close': current_bar['y17_close'],
        'ema9': current_bar['y17_ema9'],
        'vwap': current_bar['y17_vwap'],
        'ema21': current_bar['y17_ema21'], 
        'open_real': current_bar['y17_open_real'],
        'high_real': current_bar['y17_high_real'],
        'low_real': current_bar['y17_low_real'],
        'close_real': current_bar['y17_close_real']
        },

        {
        'date': 30,
        'open': current_bar['y18_open'],
        'high': current_bar['y18_high'],
        'low': current_bar['y18_low'],
        'close': current_bar['y18_close'],
        'ema9': current_bar['y18_ema9'],
        'vwap': current_bar['y18_vwap'],
        'ema21': current_bar['y18_ema21'], 
        'open_real': current_bar['y18_open_real'],
        'high_real': current_bar['y18_high_real'],
        'low_real': current_bar['y18_low_real'],
        'close_real': current_bar['y18_close_real']
        },

        {
        'date': 31,
        'open': current_bar['y19_open'],
        'high': current_bar['y19_high'],
        'low': current_bar['y19_low'],
        'close': current_bar['y19_close'],
        'ema9': current_bar['y19_ema9'],
        'vwap': current_bar['y19_vwap'],
        'ema21': current_bar['y19_ema21'], 
        'open_real': current_bar['y19_open_real'],
        'high_real': current_bar['y19_high_real'],
        'low_real': current_bar['y19_low_real'],
        'close_real': current_bar['y19_close_real']
        },

        {
        'date': 32,
        'open': current_bar['y20_open'],
        'high': current_bar['y20_high'],
        'low': current_bar['y20_low'],
        'close': current_bar['y20_close'],
        'ema9': current_bar['y20_ema9'],
        'vwap': current_bar['y20_vwap'],
        'ema21': current_bar['y20_ema21'], 
        'open_real': current_bar['y20_open_real'],
        'high_real': current_bar['y20_high_real'],
        'low_real': current_bar['y20_low_real'],
        'close_real': current_bar['y20_close_real']
        },

        {
        'date': 33,
        'open': current_bar['y21_open'],
        'high': current_bar['y21_high'],
        'low': current_bar['y21_low'],
        'close': current_bar['y21_close'],
        'ema9': current_bar['y21_ema9'],
        'vwap': current_bar['y21_vwap'],
        'ema21': current_bar['y21_ema21'], 
        'open_real': current_bar['y21_open_real'],
        'high_real': current_bar['y21_high_real'],
        'low_real': current_bar['y21_low_real'],
        'close_real': current_bar['y21_close_real']
        },

        {
        'date': 34,
        'open': current_bar['y22_open'],
        'high': current_bar['y22_high'],
        'low': current_bar['y22_low'],
        'close': current_bar['y22_close'],
        'ema9': current_bar['y22_ema9'],
        'vwap': current_bar['y22_vwap'],
        'ema21': current_bar['y22_ema21'], 
        'open_real': current_bar['y22_open_real'],
        'high_real': current_bar['y22_high_real'],
        'low_real': current_bar['y22_low_real'],
        'close_real': current_bar['y22_close_real']
        },

        {
        'date': 35,
        'open': current_bar['y23_open'],
        'high': current_bar['y23_high'],
        'low': current_bar['y23_low'],
        'close': current_bar['y23_close'],
        'ema9': current_bar['y23_ema9'],
        'vwap': current_bar['y23_vwap'],
        'ema21': current_bar['y23_ema21'], 
        'open_real': current_bar['y23_open_real'],
        'high_real': current_bar['y23_high_real'],
        'low_real': current_bar['y23_low_real'],
        'close_real': current_bar['y23_close_real']
        }        
    
    ]
    
    current_bar['current_pos'] = current_bar['x11_close']
    bars_for_evaluation = [x for x in current_bar['chart_data'] if x['date'] >= 12] 
    current_bar['is_short'] = is_short(current_bar['current_pos'], bars_for_evaluation, CURRENT_TARGET, CURRENT_STOP)
    current_bar['is_long'] = is_long(current_bar['current_pos'], bars_for_evaluation, CURRENT_TARGET, CURRENT_STOP)
    current_bar['current_date'] = current_date
    
    generated_bars.append(current_bar)
    
generated_bars_count = len(generated_bars)
generated_bars_short_count = len([x for x in generated_bars if x['is_short'] == 1])
generated_bars_long_count = len([x for x in generated_bars if x['is_long'] == 1])

print(f'(Train)Generated Bars Short:{generated_bars_short_count} ({(generated_bars_short_count / generated_bars_count) * 100}%)')
print(f'(Train)Generated Bars Long:{generated_bars_long_count} ({(generated_bars_long_count / generated_bars_count) * 100}%)')  


In [ ]:
if SHOULD_GENERATE_IMAGES:
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    generated_bars_to_visualize = [x for x in generated_bars if x['is_long'] == 1][10]
    df = pd.DataFrame(generated_bars_to_visualize['chart_data'])

    df['previous_high']=generated_bars_to_visualize['previous_high']
    df['previous_low']=generated_bars_to_visualize['previous_low']
    df['previous_close']=generated_bars_to_visualize['previous_close']
    df['previous_open']=generated_bars_to_visualize['previous_open']

    df['previous_high_real']=generated_bars_to_visualize['previous_high_real']
    df['previous_low_real']=generated_bars_to_visualize['previous_low_real']
    df['previous_close_real']=generated_bars_to_visualize['previous_close_real']
    df['previous_open_real']=generated_bars_to_visualize['previous_open_real']
    df['current_pos']=generated_bars_to_visualize['current_pos']
    
    current_start = generated_bars_to_visualize['x5_close_real']
    current_date = generated_bars_to_visualize['current_date']
    current_time = generated_bars_to_visualize['current_time']

    p = figure(x_axis_type="linear", tools=TOOLS, width=1000, title = f'WDO Candlestick, start:{current_start}, date:{current_date}, time:{current_time}')
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    w = 12*60*60*1000 # half day in ms

    inc = df.close > df.open
    dec = df.open > df.close

    p.segment(df.date, df.high, df.date, df.low, color="black")
    p.vbar(df.date[inc], 0.8, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df.date[dec], 0.8, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

    p.line(df.date, df.ema9, line_color="#00FF00", line_width = 2)
    p.line(df.date, df.ema21, line_color="#FFFF00", line_width = 2)
    p.line(df.date, df.vwap, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_high, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_close, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_open, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_low, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.current_pos, line_color="#000000", line_width = 2)

    p.background_fill_color = None
    p.border_fill_color = None  
        
    example_chart_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.chart.png")
    example_chart_full_file_name = os.path.join(DATA_OUTPUT_DIR,example_chart_file_name)
    
    export_png(p, filename=example_chart_full_file_name)
    
    show(p)

In [ ]:
raw_output_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.raw.pickle")
raw_output_full_file_name = os.path.join(DATA_OUTPUT_DIR,raw_output_file_name)
joblib.dump(generated_bars_raw, raw_output_full_file_name, True)

test_output_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.total.pickle")
test_output_full_file_name = os.path.join(DATA_OUTPUT_DIR,test_output_file_name)
joblib.dump(generated_bars, test_output_full_file_name, True)

parameter_file_name = test_output_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.parameters.pickle")
parameter_full_file_name = os.path.join(DATA_OUTPUT_DIR,test_output_file_name)
parameters = {
    "CURRENT_EXCHANGE": CURRENT_EXCHANGE,
    "TRADE_DAY_START": TRADE_DAY_START,
    "MINIMUM_TIME" : MINIMUM_TIME,
    "MAXIMUM_TIME" : MAXIMUM_TIME,
    "MINIMUM_DATE_DATAFRAME" : MINIMUM_DATE_DATAFRAME,
    "MINIMUM_DATE_TRADE" : MINIMUM_DATE_TRADE,
    "MAX_TRAIN_DATE" : MAX_TRAIN_DATE,
    "MAX_TRADE_DURATION" : MAX_TRADE_DURATION,
    "CURRENT_TARGET" : CURRENT_TARGET,
    "CURRENT_STOP" : CURRENT_STOP,
    "CURRENT_ASSET" : CURRENT_ASSET,
    "CURRENT_TIMEFRAME" : CURRENT_TIMEFRAME,
    "CURRENT_DAILY_FILE_CSV" : CURRENT_DAILY_FILE_CSV,
    "CURRENT_5MIN_FILE_CSV" : CURRENT_5MIN_FILE_CSV,
    "DECISION_BOUNDARY": DECISION_BOUNDARY,
    "CURRENT_Y_COLUMN_SHORT" : 'is_short',
    "CURRENT_Y_COLUMN_LONG" : 'is_long',
    "CURRENT_X_COLUMNS" : list(set(['x0_roc', 'x0_previous_high', 'x0_previous_low', 'x0_previous_close', 'x0_previous_open', 'x0_ema9_close', 'x0_ema21_close', 'x0_ema55_close', 
                           'x0_ema144_close', 'x0_ema233_close', 'x1_roc', 'x1_previous_high', 'x1_previous_low', 'x1_previous_close', 'x1_previous_open', 'x1_ema9_close', 
                           'x1_ema21_close', 'x1_ema55_close', 'x1_ema144_close', 'x1_ema233_close', 'x1_x0_close', 'x1_x0_high', 'x1_x0_low', 'x2_roc', 'x2_previous_high', 
                           'x2_previous_low', 'x2_previous_close', 'x2_previous_open', 'x2_ema9_close', 'x2_ema21_close', 'x2_ema55_close', 'x2_ema144_close', 'x2_ema233_close', 
                           'x2_x1_close', 'x2_x1_high', 'x2_x1_low', 'x3_roc', 'x3_previous_high', 'x3_previous_low', 'x3_previous_close', 'x3_previous_open', 'x3_ema9_close', 
                           'x3_ema21_close', 'x3_ema55_close', 'x3_ema144_close', 'x3_ema233_close', 'x3_x2_close', 'x3_x2_high', 'x3_x2_low', 'x4_roc', 'x4_previous_high', 
                           'x4_previous_low', 'x4_previous_close', 'x4_previous_open', 'x4_ema9_close', 'x4_ema21_close', 'x4_ema55_close', 'x4_ema144_close', 'x4_ema233_close', 
                           'x4_x3_close', 'x4_x3_high', 'x4_x3_low', 'x5_roc', 'x5_previous_high', 'x5_previous_low', 'x5_previous_close', 'x5_previous_open', 'x5_ema9_close', 
                           'x5_ema21_close', 'x5_ema55_close', 'x5_ema144_close', 'x5_ema233_close', 'x5_x4_close', 'x5_x4_high', 'x5_x4_low', 'x6_roc', 'x6_previous_high', 
                           'x6_previous_low', 'x6_previous_close', 'x6_previous_open', 'x6_ema9_close', 'x6_ema21_close', 'x6_ema55_close', 'x6_ema144_close', 'x6_ema233_close', 
                           'x6_x5_close', 'x6_x5_high', 'x6_x5_low', 'x7_ema9', 'x7_ema21', 'x7_ema55', 'x7_ema144', 'x7_ema233', 'x7_ema9', 'x7_ema21', 'x7_ema55', 
                           'x7_ema144', 'x7_ema233', 'x7_open', 'x7_high', 'x7_low', 'x7_close', 'x7_volume', 'x7_open', 'x7_high', 'x7_low', 'x7_close', 
                           'x7_height', 'x7_body', 'x7_roc', 'x7_previous_high', 'x7_previous_low', 'x7_previous_close', 'x7_previous_open', 'x7_ema9_close', 'x7_ema21_close', 
                           'x7_ema55_close', 'x7_ema144_close', 'x7_ema233_close', 'x7_x6_close', 'x7_x6_high', 'x7_x6_low', 'x8_roc', 'x8_previous_high', 'x8_previous_low', 
                           'x8_previous_close', 'x8_previous_open', 'x8_ema9_close', 'x8_ema21_close', 'x8_ema55_close', 'x8_ema144_close', 'x8_ema233_close', 'x8_x7_close', 
                           'x8_x7_high', 'x8_x7_low', 'x9_roc', 'x9_previous_high', 'x9_previous_low', 'x9_previous_close', 'x9_previous_open', 'x9_ema9_close', 'x9_ema21_close', 
                           'x9_ema55_close', 'x9_ema144_close', 'x9_ema233_close', 'x9_x8_close', 'x9_x8_high', 'x9_x8_low', 'x10_roc', 'x10_previous_high', 'x10_previous_low', 
                           'x10_previous_close', 'x10_previous_open', 'x10_ema9_close', 'x10_ema21_close', 'x10_ema55_close', 'x10_ema144_close', 'x10_ema233_close', 'x10_x9_close', 
                           'x10_x9_high', 'x10_x9_low', 'x11_roc', 'x11_previous_high', 'x11_previous_low', 'x11_previous_close', 'x11_previous_open', 'x11_ema9_close', 
                           'x11_ema21_close', 'x11_ema55_close', 'x11_ema144_close', 'x11_ema233_close', 'x11_x10_close', 'x11_x10_high', 'x11_x10_low','previous_close', 
                           'previous_high', 'previous_low', 'previous_open','x0_ema55', 'x0_ema144', 'x0_ema233', 'x1_ema55', 'x1_ema144', 'x1_ema233', 'x2_ema55', 'x2_ema144', 
                           'x2_ema233', 'x3_ema55', 'x3_ema144', 'x3_ema233', 'x4_ema55', 'x4_ema144', 'x4_ema233', 'x5_ema55', 'x5_ema144', 'x5_ema233', 'x6_ema55', 'x6_ema144', 
                           'x6_ema233', 'x7_ema55', 'x7_ema144', 'x7_ema233', 'x8_ema55', 'x8_ema144', 'x8_ema233', 'x9_ema55', 'x9_ema144', 'x9_ema233', 'x10_ema55', 'x10_ema144', 
                           'x10_ema233', 'x11_ema55', 'x11_ema144', 'x11_ema233', 'x0_body', 'x0_close', 'x0_ema21', 'x0_ema9', 'x0_height', 'x0_high', 'x0_low', 'x0_open', 'x0_volume',
                           'x10_body', 'x10_close', 'x10_ema21', 'x10_ema9', 'x10_height', 'x10_high', 'x10_low', 'x10_open', 'x10_volume', 'x11_body', 'x11_close', 'x11_close_slope', 
                           'x11_ema21', 'x11_ema9', 'x11_height', 'x11_high', 'x11_high_slope', 'x11_low', 'x11_low_slope', 'x11_open', 'x11_volume', 'x11_volume_slope', 'x1_body', 
                           'x1_close', 'x1_ema21', 'x1_ema9', 'x1_height', 'x1_high', 'x1_low', 'x1_open', 'x1_volume', 'x2_body', 'x2_close', 'x2_ema21', 'x2_ema9', 'x2_height', 
                           'x2_high', 'x2_low', 'x2_open', 'x2_volume', 'x3_body', 'x3_close', 'x3_ema21', 'x3_ema9', 'x3_height', 'x3_high', 'x3_low', 'x3_open', 'x3_volume', 
                           'x4_body', 'x4_close', 'x4_ema21', 'x4_ema9', 'x4_height', 'x4_high', 'x4_low', 'x4_open', 'x4_volume', 'x5_body', 'x5_close', 'x5_ema21', 'x5_ema9', 
                           'x5_height', 'x5_high', 'x5_low', 'x5_open', 'x5_volume', 'x6_body', 'x6_close', 'x6_ema21', 'x6_ema9', 'x6_height', 'x6_high', 'x6_low', 'x6_open', 
                           'x6_volume', 'x7_body', 'x7_close', 'x7_ema21', 'x7_ema9', 'x7_height', 'x7_high', 'x7_low', 'x7_open', 'x7_volume', 'x8_body', 'x8_close', 'x8_ema21', 
                           'x8_ema9', 'x8_height', 'x8_high', 'x8_low', 'x8_open', 'x8_volume', 'x9_body', 'x9_close', 'x9_ema21', 'x9_ema9', 'x9_height', 'x9_high', 'x9_low', 
                           'x9_open', 'x9_volume', 'current_bar_in_date',"x0_vwap","x1_vwap","x2_vwap","x3_vwap","x4_vwap","x5_vwap","x6_vwap","x7_vwap","x8_vwap","x9_vwap",
                           "x10_vwap","x11_vwap"]))
}

joblib.dump(parameters, parameter_full_file_name, True)

In [ ]:
generated_bars_test = pd.DataFrame(generated_bars)
generated_bars_test.tail(20)

In [ ]:
generated_bars = None 
generated_bars_raw = None 
generated_bars_test = None
generated_bars_train = None
generated_bars_total = None
data_5min = None
data_Diario = None
csv_diario = None
csv_5Min = None

gc.collect()